In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install torchmetrics
!pip install statsmodels
!pip install datasets
!pip install SimpleITK
!pip install medpy
!pip install hausdorff

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 890.6/890.6 kB 33.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.7/472.7 kB 22.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.4/52.4 MB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.3/156.3 kB 9.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for medpy: filename=MedPy-0.5.2-cp310-cp310-linux_x86_64.whl size=762834 sha256=d2d71649cd387aa1ca788b3d69d48bf83bdffef4afa540a3d2f46794fae3ad4d
  Stored in directory: /root/.cache/pip/wheels/a1/b8/63/bdf557940ec60d1b8822e73ff9fbe7727ac19f009d46b5d175
Successfully built medpy
  Preparing metadata (setup.py) ... done
  Created wheel for hausdorff: filename=hausdorff-0.2.6-py3-n

In [23]:
import numpy as np
import matplotlib.pyplot as plt
from decimal import getcontext, Decimal
import tensorflow as tf
import pandas as pd
import torch
import torchmetrics
import torchvision.transforms as transforms
from torchmetrics.functional import dice
from torchmetrics import Precision, Recall
import torch.nn.functional as F
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, jaccard_score, cohen_kappa_score
from skimage.morphology import binary_dilation, binary_erosion, disk
import numpy as np
from skimage.morphology import binary_dilation, disk
from skimage.measure import label
from scipy.ndimage import binary_erosion
from skimage.metrics import hausdorff_distance
import SimpleITK as sitk
import numpy as np
from scipy.ndimage import binary_erosion
from skimage.morphology import binary_dilation, disk
from medpy.metric.binary import hd95, __surface_distances
from medpy.metric.binary import dc, hd, jc, precision, recall
from imblearn.metrics import geometric_mean_score
from scipy.spatial.distance import directed_hausdorff
from hausdorff import hausdorff_distance
from tensorflow.keras.metrics import Precision, Recall

#**Data**

In [5]:
np.set_printoptions(precision=25)
getcontext().prec = 25
pd.options.display.float_format = '{:.25f}'.format
np.set_printoptions(precision=25)
pd.set_option('display.float_format', '{:.25f}'.format)

In [6]:
# Sample data 1: Generate random binary segmentation masks

np.random.seed(42)
y_true = np.random.randint(0, 2, (128, 128)).astype(np.int32)
y_pred = np.random.randint(0, 2, (128, 128)).astype(np.int32)

In [ ]:
# Sample data 2: binary segmentation masks

y_true = np.array([[0, 1, 1], [0, 0, 1], [1, 1, 0]])
y_pred = np.array([[0, 1, 1], [0, 1, 1], [1, 0, 0]])

In [ ]:
# Sample data 3: 3D binary segmentation masks

np.random.seed(42)
image_shape = (64, 64, 64)
y_true = np.random.randint(0, 2, size=image_shape).astype(np.float32)
y_pred = np.random.randint(0, 2, size=image_shape).astype(np.float32)
y_true_flat = y_true.reshape(-1, 1)
y_pred_flat = y_pred.reshape(-1, 1)

In [7]:
y_true_flat = y_true.flatten()
y_pred_flat = y_pred.flatten()
results = {}

#**TensorFlow Library:**

In [12]:
y_true_tf = tf.convert_to_tensor(y_true, dtype=tf.float32)
y_pred_tf = tf.convert_to_tensor(y_pred, dtype=tf.float32)

tf_accuracy = tf.keras.metrics.BinaryAccuracy()
tf_precision = tf.keras.metrics.Precision()
tf_recall = tf.keras.metrics.Recall()

tf_meaniou = tf.keras.metrics.MeanIoU(num_classes=2)
tf_iou = tf.keras.metrics.IoU(num_classes=2, target_class_ids=[1])
tf_f1 = tf.keras.metrics.F1Score(average = 'micro')

tf_accuracy.update_state(y_true_tf, y_pred_tf)
tf_precision.update_state(y_true_tf, y_pred_tf)
tf_recall.update_state(y_true_tf, y_pred_tf)
tf_meaniou.update_state(y_true_tf, y_pred_tf)
tf_iou.update_state(y_true_tf, y_pred_tf)
tf_f1.update_state(y_true_tf, y_pred_tf )



def custom_erosion_tf(mask, kernel_size=3):
    kernel = tf.ones((kernel_size, kernel_size, 1, 1), dtype=tf.float32)
    mask = tf.expand_dims(tf.expand_dims(mask, axis=0), axis=-1)  # Add batch and channel dimensions
    eroded = tf.nn.convolution(mask, filters=kernel, padding='SAME')
    eroded = tf.cast(eroded == tf.reduce_max(kernel), tf.float32)
    return tf.squeeze(eroded)  # Remove batch and channel dimensions

def custom_dilation_tf(mask, kernel_size=3):
    kernel = tf.ones((kernel_size, kernel_size, 1, 1), dtype=tf.float32)
    mask = tf.expand_dims(tf.expand_dims(mask, axis=0), axis=-1)  # Add batch and channel dimensions
    dilated = tf.nn.convolution(mask, filters=kernel, padding='SAME')
    dilated = tf.cast(dilated > 0, tf.float32)
    return tf.squeeze(dilated)  # Remove batch and channel dimensions

def extract_boundaries_tf(mask):
    """Extract boundary pixels from a binary segmentation mask using TensorFlow."""
    mask = tf.convert_to_tensor(mask, dtype=tf.float32)
    eroded_mask = custom_erosion_tf(mask, kernel_size=3)
    boundary = tf.cast(mask, tf.bool) ^ tf.cast(eroded_mask, tf.bool)
    return tf.cast(boundary, tf.float32)

def boundary_f1_score_tf(y_true, y_pred, dilation_radius=1):
    """Compute the Boundary F1 Score between the predicted and ground truth masks using TensorFlow."""
    y_true_boundary = extract_boundaries_tf(y_true)
    y_pred_boundary = extract_boundaries_tf(y_pred)

    # Dilate the boundaries to allow for some tolerance
    y_true_boundary_dilated = custom_dilation_tf(y_true_boundary, kernel_size=2*dilation_radius+1)
    y_pred_boundary_dilated = custom_dilation_tf(y_pred_boundary, kernel_size=2*dilation_radius+1)

    # Flatten the boundaries
    y_true_boundary_flat = tf.reshape(y_true_boundary_dilated, [-1])
    y_pred_boundary_flat = tf.reshape(y_pred_boundary_dilated, [-1])

    # Calculate Precision and Recall
    precision = Precision()
    recall = Recall()
    precision.update_state(y_true_boundary_flat, y_pred_boundary_flat)
    recall.update_state(y_true_boundary_flat, y_pred_boundary_flat)

    precision_value = precision.result().numpy()
    recall_value = recall.result().numpy()

    # Compute F1 score
    f1_value = 2 * (precision_value * recall_value) / (precision_value + recall_value + tf.keras.backend.epsilon())
    return f1_value
bf_score_tf = boundary_f1_score_tf(y_true, y_pred)


def pairwise_distances_tf(a, b):
    """Compute pairwise distances between each point in array a and array b."""
    a = tf.expand_dims(a, axis=1)
    b = tf.expand_dims(b, axis=0)
    return tf.reduce_sum(tf.square(a - b), axis=-1)

def hausdorff_distance_tf(y_true, y_pred):
    """Calculate Hausdorff Distance using TensorFlow."""
    y_true = tf.convert_to_tensor(y_true, dtype=tf.float32)
    y_pred = tf.convert_to_tensor(y_pred, dtype=tf.float32)

    y_true_points = tf.where(tf.equal(y_true, 1))
    y_pred_points = tf.where(tf.equal(y_pred, 1))

    if tf.size(y_true_points) == 0 or tf.size(y_pred_points) == 0:
        return tf.constant(np.inf)

    distances = pairwise_distances_tf(y_true_points, y_pred_points)
    forward_hausdorff = tf.reduce_max(tf.reduce_min(distances, axis=1))
    backward_hausdorff = tf.reduce_max(tf.reduce_min(distances, axis=0))
    hd = tf.maximum(forward_hausdorff, backward_hausdorff)
    return hd.numpy()

hd_tf = hausdorff_distance_tf(y_true, y_pred)



results['TensorFlow'] = {
    'Accuracy': tf_accuracy.result().numpy(),
    'Precision': tf_precision.result().numpy(),
    'Recall': tf_recall.result().numpy(),
    'MeanIoU': tf_meaniou.result().numpy(),
    'IoU': tf_iou.result().numpy(),
    'F1 Score': tf_f1.result().numpy(),
    'BF Score': bf_score_tf,
    'Hausdorff Distance': hd_tf
}

#**PyTorch Library:**

In [26]:
y_true_torch = torch.tensor(y_true, dtype=torch.int)
y_pred_torch = torch.tensor(y_pred, dtype=torch.int)

accuracy_metric = torchmetrics.Accuracy(task="binary")
precision_metric = torchmetrics.Precision(task="binary")
recall_metric = torchmetrics.Recall(task="binary")
f1_metric = torchmetrics.F1Score(task="binary")
iou_metric = torchmetrics.JaccardIndex(task="binary")


accuracy_metric.update(y_pred_torch, y_true_torch)
precision_metric.update(y_pred_torch, y_true_torch)
recall_metric.update(y_pred_torch, y_true_torch)
f1_metric.update(y_pred_torch, y_true_torch)
iou_metric.update(y_pred_torch, y_true_torch)

pt_accuracy = accuracy_metric.compute().item()
pt_precision = precision_metric.compute().item()
pt_recall = recall_metric.compute().item()
pt_f1 = f1_metric.compute().item()
pt_iou = iou_metric.compute().item()


def extract_boundaries_torch(mask):
    """Extract boundary pixels from a binary segmentation mask using PyTorch."""
    mask = torch.tensor(mask, dtype=torch.float32)
    eroded_mask = F.max_pool2d(mask.unsqueeze(0).unsqueeze(0), kernel_size=3, stride=1, padding=1).squeeze()
    boundary = (mask != eroded_mask).float()
    return boundary

def custom_dilation_torch(mask, kernel_size=3):
    """Dilate the boundary pixels."""
    kernel = torch.ones((1, 1, kernel_size, kernel_size), dtype=torch.float32)
    mask = mask.unsqueeze(0).unsqueeze(0)  # Add batch and channel dimensions
    dilated = F.conv2d(mask, kernel, padding=kernel_size // 2).squeeze()
    dilated = (dilated > 0).float()
    return dilated

def boundary_f1_score_torch(y_true, y_pred, dilation_radius=1):
    """Compute the Boundary F1 Score between the predicted and ground truth masks using PyTorch."""
    y_true_boundary = extract_boundaries_torch(y_true)
    y_pred_boundary = extract_boundaries_torch(y_pred)

    # Dilate the boundaries to allow for some tolerance
    y_true_boundary_dilated = custom_dilation_torch(y_true_boundary, kernel_size=2*dilation_radius+1)
    y_pred_boundary_dilated = custom_dilation_torch(y_pred_boundary, kernel_size=2*dilation_radius+1)

    # Flatten the boundaries
    y_true_boundary_flat = y_true_boundary_dilated.flatten()
    y_pred_boundary_flat = y_pred_boundary_dilated.flatten()

    # Calculate Precision and Recall
    precision_metric = Precision('binary')
    recall_metric = Recall('binary')

    precision_metric.update(y_pred_boundary_flat, y_true_boundary_flat)
    recall_metric.update(y_pred_boundary_flat, y_true_boundary_flat)

    precision_value = precision_metric.compute().item()
    recall_value = recall_metric.compute().item()

    # Compute F1 score
    f1_value = 2 * (precision_value * recall_value) / (precision_value + recall_value + 1e-6)
    return f1_value

bf_score_torch = boundary_f1_score_torch(y_true, y_pred)



def hausdorff_distance_torch(y_true, y_pred):
    """Calculate Hausdorff Distance using PyTorch."""
    y_true = torch.tensor(y_true, dtype=torch.float32)
    y_pred = torch.tensor(y_pred, dtype=torch.float32)
    y_true_points = torch.nonzero(y_true, as_tuple=False).float()
    y_pred_points = torch.nonzero(y_pred, as_tuple=False).float()

    if y_true_points.size(0) == 0 or y_pred_points.size(0) == 0:
        return float('inf')

    dists = torch.cdist(y_true_points.unsqueeze(0), y_pred_points.unsqueeze(0)).squeeze(0)
    forward_hausdorff = torch.max(torch.min(dists, dim=1)[0])
    backward_hausdorff = torch.max(torch.min(dists, dim=0)[0])
    hd = torch.max(forward_hausdorff, backward_hausdorff)
    return hd.item()
hd_torch = hausdorff_distance_torch(y_true, y_pred)

results['PyTorch/torchmetrics'] = {
    'Accuracy': pt_accuracy,
    'Precision': pt_precision,
    'Recall': pt_recall,
    'F1 Score': pt_f1,
    'IoU': pt_iou,
    'Dice Coefficient': dice(y_pred_torch, y_true_torch).item(),
    'BF Score': bf_score_torch,
    'Hausdorff Distance': hd_torch
}

#**Scikit Learn Library:**

In [14]:
def extract_boundaries_skl(mask):
    """ Extract boundary pixels from a binary segmentation mask. """
    structuring_element = disk(1)
    eroded_mask = binary_erosion(mask, structuring_element)
    boundary = mask ^ eroded_mask
    return boundary

def boundary_f1_score_skl(y_true, y_pred, dilation_radius=1):
    """ Compute the Boundary F1 Score between the predicted and ground truth masks. """
    y_true_boundary = extract_boundaries_skl(y_true)
    y_pred_boundary = extract_boundaries_skl(y_pred)

    # Dilate the boundaries to allow for some tolerance
    structuring_element = disk(dilation_radius)
    y_true_boundary_dilated = binary_dilation(y_true_boundary, structuring_element)
    y_pred_boundary_dilated = binary_dilation(y_pred_boundary, structuring_element)

    # Flatten the boundaries
    y_true_boundary_flat = y_true_boundary_dilated.flatten()
    y_pred_boundary_flat = y_pred_boundary_dilated.flatten()

    # Compute F1 score
    bf_score = f1_score(y_true_boundary_flat, y_pred_boundary_flat)
    return bf_score

bf_score_skl = boundary_f1_score_skl(y_true, y_pred)

sklearn_accuracy = accuracy_score(y_true_flat, y_pred_flat)
sklearn_precision = precision_score(y_true_flat, y_pred_flat)
sklearn_recall = recall_score(y_true_flat, y_pred_flat)
sklearn_f1 = f1_score(y_true_flat, y_pred_flat)
sklearn_jaccard = jaccard_score(y_true_flat, y_pred_flat)
sklearn_kappa = cohen_kappa_score(y_true_flat, y_pred_flat)

results['Scikit-Learn'] = {
    'Accuracy': sklearn_accuracy,
    'Precision': sklearn_precision,
    'Recall': sklearn_recall,
    'F1 Score': sklearn_f1,
    'IoU': sklearn_jaccard,
    'Kappa': sklearn_kappa,
    'BF Score': bf_score_skl
}

#**Sicikit Image Library:**

In [15]:
def extract_boundaries_ski(mask):
    """ Extract boundary pixels from a binary segmentation mask. """
    structuring_element = disk(1)
    eroded_mask = binary_erosion(mask, structure=structuring_element)
    boundary = mask ^ eroded_mask
    return boundary

def boundary_f1_score_ski(pred, gt, dilation_radius=1):
    """ Compute the Boundary F1 Score between the predicted and ground truth masks. """
    pred_boundary = extract_boundaries_ski(pred)
    gt_boundary = extract_boundaries_ski(gt)

    # Dilate the boundaries to allow for tolerance in matching
    structuring_element = disk(dilation_radius)
    pred_boundary_dilated = binary_dilation(pred_boundary, structuring_element)
    gt_boundary_dilated = binary_dilation(gt_boundary, structuring_element)

    # True positives
    true_positives = np.sum(pred_boundary & gt_boundary_dilated)
    # False positives
    false_positives = np.sum(pred_boundary & ~gt_boundary_dilated)
    # False negatives
    false_negatives = np.sum(gt_boundary & ~pred_boundary_dilated)

    if true_positives + false_positives == 0 or true_positives + false_negatives == 0:
        return 0.0

    # Precision and recall
    precision = true_positives / (true_positives + false_positives)
    recall = true_positives / (true_positives + false_negatives)

    # F1 Score
    bf_score = 2 * (precision * recall) / (precision + recall)
    return bf_score

bf_score_ski = boundary_f1_score_ski(y_pred, y_true)
hd_ski = hausdorff_distance(y_true, y_pred)

results['Scikit-Image'] = {
    'BF Score': bf_score_ski,
    'Hausdorff Distance': hd_ski
}


#**SimpleITK Library:**

In [16]:
def extract_boundaries_itk(mask):
    """ Extract boundary pixels from a binary segmentation mask using SimpleITK. """
    mask_sitk = sitk.GetImageFromArray(mask.astype(np.uint8))
    eroded_mask_sitk = sitk.BinaryErode(mask_sitk, (1,1))
    eroded_mask = sitk.GetArrayFromImage(eroded_mask_sitk)
    boundary = mask ^ eroded_mask
    return boundary

def boundary_f1_score_itk(pred, gt, dilation_radius=1):
    """ Compute the Boundary F1 Score between the predicted and ground truth masks using SimpleITK. """
    pred_boundary = extract_boundaries_itk(pred)
    gt_boundary = extract_boundaries_itk(gt)

    # Dilate the boundaries to allow for tolerance in matching
    pred_boundary_sitk = sitk.GetImageFromArray(pred_boundary.astype(np.uint8))
    gt_boundary_sitk = sitk.GetImageFromArray(gt_boundary.astype(np.uint8))

    pred_boundary_dilated = sitk.BinaryDilate(pred_boundary_sitk, (dilation_radius, dilation_radius))
    gt_boundary_dilated = sitk.BinaryDilate(gt_boundary_sitk, (dilation_radius, dilation_radius))

    pred_boundary_dilated = sitk.GetArrayFromImage(pred_boundary_dilated)
    gt_boundary_dilated = sitk.GetArrayFromImage(gt_boundary_dilated)

    # True positives
    true_positives = np.sum(pred_boundary & gt_boundary_dilated)
    # False positives
    false_positives = np.sum(pred_boundary & ~gt_boundary_dilated)
    # False negatives
    false_negatives = np.sum(gt_boundary & ~pred_boundary_dilated)

    if true_positives + false_positives == 0 or true_positives + false_negatives == 0:
        return 0.0

    # Precision and recall
    precision = true_positives / (true_positives + false_positives)
    recall = true_positives / (true_positives + false_negatives)

    # F1 Score
    bf_score = 2 * (precision * recall) / (precision + recall)
    return bf_score
bf_score_itk = boundary_f1_score_itk(y_pred, y_true)

def calculate_iou_sitk(y_true, y_pred):
    """Calculate Intersection over Union (IoU) using SimpleITK."""
    # Convert numpy arrays to SimpleITK images
    y_true_sitk = sitk.GetImageFromArray(y_true)
    y_pred_sitk = sitk.GetImageFromArray(y_pred)

    # Calculate the intersection and union
    intersection = sitk.And(y_true_sitk, y_pred_sitk)
    union = sitk.Or(y_true_sitk, y_pred_sitk)

    # Convert back to numpy arrays
    intersection_array = sitk.GetArrayFromImage(intersection)
    union_array = sitk.GetArrayFromImage(union)

    # Compute IoU
    iou = np.sum(intersection_array) / np.sum(union_array)
    return iou

iou_value_sitk = calculate_iou_sitk(y_true, y_pred)


y_true_sitk = sitk.GetImageFromArray(y_true.astype(np.uint8))
y_pred_sitk = sitk.GetImageFromArray(y_pred.astype(np.uint8))

hausdorff_filter = sitk.HausdorffDistanceImageFilter()
hausdorff_filter.Execute(y_true_sitk, y_pred_sitk)
hd_itk = hausdorff_filter.GetHausdorffDistance()

import numpy as np
import SimpleITK as sitk

def dice_coefficient_itk(y_true, y_pred):
    """Calculate Dice Coefficient using SimpleITK."""
    # Convert numpy arrays to SimpleITK images
    y_true_sitk = sitk.GetImageFromArray(y_true)
    y_pred_sitk = sitk.GetImageFromArray(y_pred)

    # Calculate Dice Coefficient
    dice_filter = sitk.LabelOverlapMeasuresImageFilter()
    dice_filter.Execute(y_true_sitk, y_pred_sitk)
    dice_value = dice_filter.GetDiceCoefficient()
    return dice_value

dice_itk = dice_coefficient_itk(y_true, y_pred)

results['SimpleITK'] = {
    'BF Score': bf_score_itk,
    'IoU': iou_value_sitk,
    'Hausdorff Distance': hd_itk,
    'Dice Coefficient': dice_itk
}

#**Medpy Library:**

In [17]:
def extract_boundaries_med(mask):
    """ Extract boundary pixels from a binary segmentation mask. """
    structuring_element = disk(1)
    eroded_mask = binary_erosion(mask, structuring_element)
    boundary = mask ^ eroded_mask
    return boundary

def compute_surface_distances_med(pred_boundary, gt_boundary):
    """ Compute the surface distances using MedPy. """
    return __surface_distances(pred_boundary, gt_boundary)

def boundary_f1_score_med(pred, gt, dilation_radius=1):
    """ Compute the Boundary F1 Score between the predicted and ground truth masks using MedPy. """
    pred_boundary = extract_boundaries_med(pred)
    gt_boundary = extract_boundaries_med(gt)

    # Dilate the boundaries to allow for tolerance in matching
    structuring_element = disk(dilation_radius)
    pred_boundary_dilated = binary_dilation(pred_boundary, structuring_element)
    gt_boundary_dilated = binary_dilation(gt_boundary, structuring_element)

    # Compute surface distances using MedPy
    pred_to_gt_distances = compute_surface_distances_med(pred_boundary, gt_boundary_dilated)
    gt_to_pred_distances = compute_surface_distances_med(gt_boundary, pred_boundary_dilated)

    # # True positives
    true_positives = np.sum(pred_boundary & gt_boundary_dilated)
    # False positives
    false_positives = np.sum(pred_boundary & ~gt_boundary_dilated)
    # False negatives
    false_negatives = np.sum(gt_boundary & ~pred_boundary_dilated)
    #false_negatives = np.sum(gt_boundary_dilated & ~pred_boundary)

    if true_positives + false_positives == 0 or true_positives + false_negatives == 0:
        return 0.0

    # Precision and recall
    precision = true_positives / (true_positives + false_positives)
    recall = true_positives / (true_positives + false_negatives)

    # F1 Score
    bf_score = 2 * (precision * recall) / (precision + recall)
    return bf_score

bf_score_med = boundary_f1_score_med(y_pred, y_true)
dc_medpy = dc(y_true, y_pred)
hausdorff_distance_med = hd(y_true, y_pred)

# Compute Jaccard Index (IoU)
iou_medpy = jc(y_true, y_pred)

precision_medpy = precision(y_pred, y_true)
recall_medpy = recall(y_pred, y_true)

f1_medpy = 2 * (precision_medpy * recall_medpy) / (precision_medpy + recall_medpy)

results['Medpy'] = {
    'BF Score': bf_score_med,
    'Dice Coefficient': dc_medpy,
    'Hausdorff Distance' : hausdorff_distance_med,
    'IoU': iou_medpy,
    'Precision': precision_medpy,
    'Recall': recall_medpy,
    'F1 Score': f1_medpy
}

#**Imbalanced Learn Library:**

In [18]:
results['Imbalanced-learn'] = {
    'Geometric Mean': geometric_mean_score(y_true_flat, y_pred_flat)
}

#**Numpy Library:**

In [19]:
def calculate_iou(y_true, y_pred):
    intersection = np.logical_and(y_true, y_pred)
    union = np.logical_or(y_true, y_pred)
    return np.sum(intersection) / np.sum(union)

# Function to calculate Dice coefficient
def calculate_dice(y_true, y_pred):
    intersection = np.logical_and(y_true, y_pred)
    return 2 * np.sum(intersection) / (np.sum(y_true) + np.sum(y_pred))

# Function to calculate Pixel Accuracy
def calculate_pixel_accuracy(y_true, y_pred):
    return np.sum(y_true == y_pred) / y_true.size

y_true_points = np.column_stack(np.where(y_true == 1))
y_pred_points = np.column_stack(np.where(y_pred == 1))
hd_scipy = max(directed_hausdorff(y_true_points, y_pred_points)[0],
                directed_hausdorff(y_pred_points, y_true_points)[0])

results['Numpy'] = {
    'IoU': calculate_iou(y_true, y_pred),
    'Dice Coefficient': calculate_dice(y_true, y_pred),
    'Accuracy': calculate_pixel_accuracy(y_true, y_pred),
    'Hausdorff Distance': hd_scipy
}

#**hausdorff Library:**

In [20]:
distance_haus = hausdorff_distance(y_pred, y_true)

results['hausdorff'] = {
    'Hausdorff Distance': distance_haus
}

In [21]:
results_df = pd.DataFrame(results).T
#results_df.index.name = 'Library'  # Set index name for better readability
#results_df.reset_index(inplace=True)  # Reset index to make 'Library' a column
#results_df = results_df.rename_axis(None, axis=1)  # Remove axis name for columns
print(results_df)

                                    Accuracy                   Precision  \
TensorFlow       0.4956054687500000000000000 0.4922054409980773925781250   
Scikit-Learn     0.4956054687500000000000000 0.4922054380664652728505359   
Scikit-Image                             NaN                         NaN   
SimpleITK                                NaN                         NaN   
Medpy                                    NaN 0.4922054380664652728505359   
Imbalanced-learn                         NaN                         NaN   
Numpy            0.4956054687500000000000000                         NaN   
hausdorff                                NaN                         NaN   

                                      Recall                     MeanIoU  \
TensorFlow       0.5006760954856872558593750 0.3294377326965332031250000   
Scikit-Learn     0.5006760909649662361076139                         NaN   
Scikit-Image                             NaN                         NaN   
SimpleITK  

# Segmentation Model